# Preparação da tabela de OP

### Importando as bibliotecas necessárias

In [ ]:
import numpy as np
import pandas as pd
from resolve_path import ajuste_path, read_input

### Lendo o dataframe de OP

In [ ]:
pathUtil = "data/util/"
pathUtil = ajuste_path(pathUtil)

df_original = pd.read_csv(
    pathUtil + "os/operacoes_preprocessada.csv", sep="@", on_bad_lines='skip')

df_coord = pd.read_csv(pathUtil + "local_coordenada.csv", sep=";")

df_copy_ori = df_original.copy()
df_copy_coord = df_coord.copy()

In [ ]:
df_original = df_copy_ori
df_coord = df_copy_coord

print(df_original.info())
print(df_coord.info())

### Adiciona coluna TAM (Tipo de atividade)

Leitura das tabelas ZPM e Suporte-TAM

In [ ]:
df_tam = read_input("Suporte-TAM.csv")

df_zpm = read_input("ZPM_DATAS_ORDEM_Executadas-2020-até-28jun24.csv")

Observação: 
Todas as ordens presentes na tabela IW47 estão contidas na tabela ZPM.

Mas atualmente, (06/08/24), existem 9196 ordens de 80809 ordens totais na tabela ZPM que não estão contidas na IW47. 

In [ ]:
# Converte as colunas de interesse para string
df_original["ordem"] = df_original["ordem"].astype(str)
df_zpm["Nº ordem"] = df_zpm["Nº ordem"].astype(str)
df_tam["TAM"] = df_tam["TAM"].astype(str)
df_zpm["Tipo de atividade"] = df_zpm["Tipo de atividade"].astype(str)

# Adicionando a coluna 'Denominação TAM' do df_tam ao df_zpm
df_zpm_tam = pd.merge(
    df_zpm, df_tam, left_on=['Tipo de Ordem', 'Tipo de atividade'], right_on=['Tipo Ordem', 'TAM'], how='left')

# Adiciona as colunas 'tam' e 'Denominação TAM' do df_zpm_tam ao df_original
df_original = pd.merge(df_original, df_zpm_tam[[
                       "Nº ordem", "TAM", "Denominação TAM"]], left_on='ordem', right_on='Nº ordem', how='left')

In [ ]:
df_original.info()

### Removendo colunas que não serão usadas

In [ ]:
df_original = df_original[
    [
        'ordem',
        'operacao',
        'suboperacao',
        'contador',
        'confirmacao',
        'no_pessoal',
        'tipo_ordem',
        'TAM',
        'Denominação TAM',
        'txt.breve_operacao',
        'local_de_instalacao',
        'trabalho_real',
        'unid.trabalho_(real)',
        'trabalho_(planejado)',
        'unidade_trab.(plan.)',
        'data_do_inicio_real',
        'data_do_fim_real',
        'data_fim_prevista',
        'estornado',
        'confirmacao_estorn.'
    ]
].reset_index()

df_original.rename(
    columns={'TAM': 'tam', 'Denominação TAM': 'denominacao_tam'}, inplace=True)

df_original.info()

### Tirando OP com HH igual a 0 e removendo confirmações extornadas

In [ ]:
# df_original = df_original.loc[
#     df_original["Trabalho real"] != 0
# ]

df_original = df_original[df_original['estornado'] != 'X']
df_original = df_original[df_original['confirmacao_estorn.'] == 0]

colunas_estorno = [
    'estornado',
    'confirmacao_estorn.'
]

df_original.drop(columns=colunas_estorno, axis=1, inplace=True)

### Remove linhas com 'txt.breve operação' igual a 'relatorio' e 'planejamento' pois não oferecem risco

In [ ]:
# Remove linhas com 'txt.breve operação' igual a 'relatorio' e 'planejamento'
df_original = df_original[~df_original["txt.breve_operacao"].isin(
    ["relatorio", "planejamento"])]

### Removendo OP sem Local de Instalação

In [ ]:
df_original = df_original.loc[df_original["local_de_instalacao"] != ""]
df_original['local_de_instalacao'] = df_original['local_de_instalacao'].dropna()

### Reduzindo a quantidade de siglas do Local de Instalação

In [ ]:
def local(linha):
    string = str(linha)
    parts = string.split("-")
    tam = len(parts)
    if tam > 3:
        if parts[1] == 'L':
            # if tam == 5:
            #     result = "-".join(parts[:5])
            # if parts[5].startswith('V0'):
            #     return "-".join(parts[:6])
            result = string
        else:
            result = "-".join(parts[:3])
    else:
        result = string

    return result


df_original["local_de_instalacao"] = df_original["local_de_instalacao"].apply(
    local)
df_coord["local_de_instalacao"] = df_coord["local_de_instalacao"].apply(local)

### Removendo OP com Local de instalação sem coordenada



In [ ]:
print(df_original["local_de_instalacao"].nunique())
print(df_coord["local_de_instalacao"].nunique())

locais_nao_presentes = df_coord[
    ~df_coord["local_de_instalacao"].isin(df_original["local_de_instalacao"])
]
print("Locais nao presentes: ")
print(locais_nao_presentes["local_de_instalacao"].unique())

operacoes_nao_presentes = df_original[~df_original["local_de_instalacao"].isin(
    df_coord["local_de_instalacao"])]

print("operacoes nao presentes:")
print(operacoes_nao_presentes["local_de_instalacao"].unique())

df_original = df_original[
    df_original["local_de_instalacao"].isin(df_coord["local_de_instalacao"])
]

print(df_original["local_de_instalacao"].nunique())

### Adicionando coordenadas aos Locais de instalação

In [ ]:
# df_original["Latitude"] = df_coord[]
for li in df_original["local_de_instalacao"].unique():
    try:
        df_original.loc[df_original['local_de_instalacao'] == li,
                        'latitude'] = df_coord.loc[df_coord['local_de_instalacao'] == li, 'latitude'].values[0]
        df_original.loc[df_original['local_de_instalacao'] == li,
                        'longitude'] = df_coord.loc[df_coord['local_de_instalacao'] == li, 'longitude'].values[0]
    except:
        pass

df_original

### Reduzindo os Txt.breve Operacao e agrupando os com menos frequência em Outros

In [ ]:
def top_n(df, n, coluna):
    top = df[coluna].value_counts().head(n).index

    df.loc[~df[coluna].isin(top), coluna] = "outros"

    return df


df_original = top_n(df_original, 5, "txt.breve_operacao")

### Agrupando TAMs com o mesmo significado

In [ ]:
df_original.loc[df_original['denominacao_tam'] ==
                'Melhorias', 'denominacao_tam'] = 'MELHORIA'
df_original.loc[df_original['tam'] == 470, 'tam'] = 260

df_original.loc[df_original['denominacao_tam'] ==
                'REPARO DE FALHA', 'denominacao_tam'] = 'REPARO DEFEITO'
df_original.loc[df_original['tam'] == 300, 'tam'] = 290

df_original.loc[df_original['denominacao_tam'] ==
                'Ensaios gerais', 'denominacao_tam'] = 'ENSAIOS'
df_original.loc[df_original['tam'] == 410, 'tam'] = 150

### Reduzindo os TAMs e agrupando os com menos frequência em Outros

In [ ]:
df_original = top_n(df_original, 10, "denominacao_tam")

### Criando Data de início e fim

In [ ]:
# df_original["data_inicio"] = df_original["data_do_inicio_real"]
# df_original["data_fim"] = df_original["data_do_fim_real"]
df_original.rename(columns={'data_do_inicio_real': 'data_inicio',
                            'data_do_fim_real': 'data_fim'},
                   inplace=True)

colunas_data = [
    "data_fim_prevista"
]

df_original.drop(columns=colunas_data, axis=1, inplace=True)

### Criando a coluna do HH final do agrupamento

In [ ]:
df_original.rename(columns={'trabalho_real': 'hh'},
                   inplace=True)

colunas_trabalho = [
    "trabalho_(planejado)",
]

df_original.drop(columns=colunas_trabalho, axis=1, inplace=True)

### Trocando os HH em minutos para horas

In [ ]:
df_original.loc[df_original['unid.trabalho_(real)'] == "MIN", "hh"] /= 60
df_original.loc[df_original['unid.trabalho_(real)']
                == "MIN", "Unid.trabalho (real)"] = "HR"

### Montando a tabela com as colunas necessárias

In [ ]:
df_cortado = df_original[
    [
        "ordem",
        "operacao",
        'suboperacao',
        'contador',
        'no_pessoal',
        "confirmacao",
        "txt.breve_operacao",
        "local_de_instalacao",
        "latitude",
        "longitude",
        "data_inicio",
        "data_fim",
        "hh",
        "tam",
        "denominacao_tam",
    ]
]

df = df_cortado.sort_values(by=["data_inicio", "local_de_instalacao"])

print(df.head(5))

### Exportando o DataFrame

In [ ]:
df.to_csv(pathUtil + "os/operacoes_preparado.csv", index=False)